# Network Traffic Anomaly Detection
# Data Preprocessing, Visualization, and Model Training

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import DBSCAN
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, classification_report
from sklearn.decomposition import PCA
import joblib
import warnings
warnings.filterwarnings('ignore')

## 1. Data Loading and Initial Exploration

In [2]:
print("Loading dataset...")

df = pd.read_csv("Dataset/Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv")

Loading dataset...


In [3]:
print(f"Dataset loaded. Shape: {df.shape}")

Dataset loaded. Shape: (170366, 79)


In [4]:
# Display sample data
print("\nSample data:")
df.head()


Sample data:


,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,389,113095465,48,24,9668,10012,403,0,201.416667,203.548293,...,32,203985.500,5.758373e+05,1629110,379,13800000.0,4.277541e+06,16500000,6737603,BENIGN
1,389,113473706,68,40,11364,12718,403,0,167.117647,171.919413,...,32,178326.875,5.034269e+05,1424245,325,13800000.0,4.229413e+06,16500000,6945512,BENIGN
2,0,119945515,150,0,0,0,0,0,0.000000,0.000000,...,0,6909777.333,1.170000e+07,20400000,6,24400000.0,2.430000e+07,60100000,5702188,BENIGN
3,443,60261928,9,7,2330,4221,1093,0,258.888889,409.702161,...,20,0.000,0.000000e+00,0,0,0.0,0.000000e+00,0,0,BENIGN
4,53,269,2,2,102,322,51,51,51.000000,0.000000,...,32,0.000,0.000000e+00,0,0,0.0,0.000000e+00,0,0,BENIGN


In [5]:
# Check data types and missing values
print("\nData types:")
print(df.dtypes)


Data types:
 Destination Port                int64
 Flow Duration                   int64
 Total Fwd Packets               int64
 Total Backward Packets          int64
Total Length of Fwd Packets      int64
                                ...   
Idle Mean                      float64
 Idle Std                      float64
 Idle Max                        int64
 Idle Min                        int64
 Label                          object
Length: 79, dtype: object


In [6]:
print("\nMissing values per column:")
print(df.isnull().sum())


Missing values per column:
 Destination Port              0
 Flow Duration                 0
 Total Fwd Packets             0
 Total Backward Packets        0
Total Length of Fwd Packets    0
                              ..
Idle Mean                      0
 Idle Std                      0
 Idle Max                      0
 Idle Min                      0
 Label                         0
Length: 79, dtype: int64


# 2. Data Preprocessing

In [7]:
# Handle missing values
print("Handling missing values...")
# Replace missing values in numeric columns with mean
numeric_cols = df.select_dtypes(include=[np.number]).columns
df[numeric_cols] = df[numeric_cols].fillna(df[numeric_cols].mean())

Handling missing values...


In [8]:
# Remove rows with any remaining missing values
df = df.dropna()
print(f"Shape after handling missing values: {df.shape}")

Shape after handling missing values: (170366, 79)


In [9]:
# Convert non-numeric columns if any
categorical_cols = df.select_dtypes(include=['object']).columns.tolist()
if 'Label' in categorical_cols:
    categorical_cols.remove('Label')  # Keep Label as is

print(f"Converting {len(categorical_cols)} categorical columns...")
for col in categorical_cols:
    df[col] = pd.factorize(df[col])[0]

Converting 1 categorical columns...


In [10]:
# Check for infinite values and replace with column max
print("Handling infinite values...")
df.replace([np.inf, -np.inf], np.nan, inplace=True)
df.fillna(df.max(), inplace=True)

Handling infinite values...


In [11]:
# Extract labels
y = df[' Label']
# Binary classification: normal vs anomaly
y_binary = y.apply(lambda x: 0 if x == 'BENIGN' else 1)

In [12]:
# Remove label columns from features
X = df.drop(' Label', axis=1)

In [13]:
# Feature scaling
print("Scaling features...")
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

Scaling features...


In [14]:
# Save preprocessed data and scaler
print("Saving preprocessed data...")
joblib.dump(scaler, 'scaler.pkl')
np.save('X_scaled.npy', X_scaled)
np.save('y_binary.npy', y_binary)
y.to_pickle('y_full.pkl')

Saving preprocessed data...


## 3. Exploratory Data Analysis and Visualization

In [15]:
# Reduce dimensions for visualization with PCA
print("Performing PCA for visualization...")
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled)

Performing PCA for visualization...


In [16]:
# Plot PCA results with normal vs attack labels
plt.figure(figsize=(12, 8))
sns.scatterplot(x=X_pca[:, 0], y=X_pca[:, 1], hue=y_binary, palette={0: 'blue', 1: 'red'}, alpha=0.6)
plt.title('PCA Visualization - Normal vs Attack Traffic')
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.legend(labels=['Normal', 'Attack'])
plt.savefig('PCA_visualization.png')
plt.close()

In [17]:
# Distribution of some key features
print("Visualizing feature distributions...")
key_features = ['Flow Duration', 'Total Fwd Packets', 'Total Backward Packets', 
                'Total Length of Fwd Packets', 'Flow Packets/s']

plt.figure(figsize=(15, 12))
for i, feature in enumerate(key_features):
    if feature in X.columns:
        plt.subplot(3, 2, i+1)
        sns.histplot(data=df, x=feature, hue=' Label', element='step', bins=50, common_norm=False)
        plt.title(f'Distribution of {feature}')
        plt.yscale('log')
plt.tight_layout()
plt.savefig('feature_distributions.png')
plt.close()

Visualizing feature distributions...


In [18]:
# Correlation matrix for top features
print("Creating correlation matrix...")
plt.figure(figsize=(16, 12))
top_features = X.columns[:15]  # Top 15 features
correlation_matrix = df[top_features].corr()
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', linewidths=0.5)
plt.title('Correlation Matrix of Top Features')
plt.tight_layout()
plt.savefig('correlation_matrix.png')
plt.close()

Creating correlation matrix...


## 4. Clustering with DBSCAN for Anomaly Detection

In [19]:
# Split data for training
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_binary, test_size=0.3, random_state=42)

In [20]:
# Apply DBSCAN
print("Applying DBSCAN...")
dbscan = DBSCAN(eps=0.5, min_samples=5, n_jobs=-1)
dbscan_labels = dbscan.fit_predict(X_train)

Applying DBSCAN...


In [21]:
# Analyze DBSCAN results
print(f"Number of clusters found: {len(np.unique(dbscan_labels)) - (1 if -1 in dbscan_labels else 0)}")
print(f"Number of noise points: {(dbscan_labels == -1).sum()}")

Number of clusters found: 733
Number of noise points: 16453


In [22]:
# Visualize clusters
pca_train = pca.transform(X_train)
plt.figure(figsize=(12, 8))
sns.scatterplot(x=pca_train[:, 0], y=pca_train[:, 1], hue=dbscan_labels, palette='bright', legend='full')
plt.title('DBSCAN Clustering Results')
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.savefig('DBSCAN_clusters.png')
plt.close()

#### *** UPDATED: Remove noise points before model training ***

In [23]:
# print("\n--- Removing Noise Points ---")
# # Get indices of non-noise points
# non_noise_indices = np.where(dbscan_labels != -1)[0]
# print(f"Keeping {len(non_noise_indices)} points after removing noise")

In [24]:
# # Filter out noise points
# X_train_filtered = X_train[non_noise_indices]
# y_train_filtered = y_train.iloc[non_noise_indices]

In [25]:
# # Visualize filtered data
# pca_train_filtered = pca_train[non_noise_indices]
# plt.figure(figsize=(12, 8))
# sns.scatterplot(x=pca_train_filtered[:, 0], y=pca_train_filtered[:, 1], 
#                 hue=dbscan_labels[non_noise_indices], palette='bright', legend='full')
# plt.title('DBSCAN Clusters After Removing Noise Points')
# plt.xlabel('Principal Component 1')
# plt.ylabel('Principal Component 2')
# plt.show()

In [26]:
# # Compare distribution of classes before and after removing noise
# plt.figure(figsize=(10, 6))
# plt.subplot(1, 2, 1)
# sns.countplot(x=y_train)
# plt.title('Class Distribution Before Removing Noise')
# plt.xticks([0, 1], ['Normal', 'Attack'])

# plt.subplot(1, 2, 2)
# sns.countplot(x=y_train_filtered)
# plt.title('Class Distribution After Removing Noise')
# plt.xticks([0, 1], ['Normal', 'Attack'])
# plt.tight_layout()
# plt.show()

## 5. Supervised Learning with Random Forest

In [27]:
# Add DBSCAN cluster as a feature
X_train_with_cluster = np.column_stack((X_train, dbscan_labels))
X_test_with_cluster = np.column_stack((X_test, dbscan.fit_predict(X_test)))

In [28]:
# Train Random Forest on filtered data
print("Training Random Forest model...")
rf_model = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
rf_model.fit(X_train_with_cluster, y_train)

Training Random Forest model...


RandomForestClassifier(n_jobs=-1, random_state=42)

In [29]:
# Save the model
joblib.dump(rf_model, 'random_forest_model.pkl')

['random_forest_model.pkl']

In [30]:
# Make predictions
y_pred = rf_model.predict(X_test_with_cluster)

## 6. Model Evaluation

In [31]:
# Calculate metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")

Accuracy: 1.0000
Precision: 1.0000
Recall: 1.0000


In [32]:
# Display confusion matrix
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title('Confusion Matrix')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.savefig('confusion_matrix.png')
plt.close()

In [33]:
# Display classification report
print("\nClassification Report:")
print(classification_report(y_test, y_pred))


Classification Report:
              precision    recall  f1-score   support

           1       1.00      1.00      1.00     51110

    accuracy                           1.00     51110
   macro avg       1.00      1.00      1.00     51110
weighted avg       1.00      1.00      1.00     51110



## 7. Feature Importance

In [34]:
# Get feature importance from Random Forest
feature_names = list(X.columns) + ['DBSCAN_Cluster']
feature_importances = rf_model.feature_importances_

In [35]:
# Sort features by importance
indices = np.argsort(feature_importances)[-20:]  # Top 20 features
plt.figure(figsize=(12, 8))
plt.barh(range(len(indices)), feature_importances[indices], align='center')
plt.yticks(range(len(indices)), [feature_names[i] for i in indices])
plt.title('Top 20 Feature Importances')
plt.xlabel('Relative Importance')
plt.tight_layout()
plt.savefig('feature_importances.png')
plt.close()

## 8. Performance Comparison (with vs without noise removal)

In [36]:
# Train a model on the original data (with noise points)
print("Training comparison model on unfiltered data...")
rf_model_with_noise = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
X_train_with_noise_cluster = np.column_stack((X_train, dbscan_labels))
rf_model_with_noise.fit(X_train_with_noise_cluster, y_train)

Training comparison model on unfiltered data...


RandomForestClassifier(n_jobs=-1, random_state=42)

In [37]:
# Make predictions
y_pred_with_noise = rf_model_with_noise.predict(X_test_with_cluster)

In [38]:
# Calculate metrics for model trained with noise
accuracy_with_noise = accuracy_score(y_test, y_pred_with_noise)
precision_with_noise = precision_score(y_test, y_pred_with_noise, average='weighted')
recall_with_noise = recall_score(y_test, y_pred_with_noise, average='weighted')

In [39]:
# Print comparison
print("\nPerformance Comparison:")
print(f"{'Metric':<20} {'With Noise':<15} {'Without Noise':<15} {'Difference':<15}")
print(f"{'-'*60}")
print(f"{'Accuracy':<20} {accuracy_with_noise:<15.4f} {accuracy:<15.4f} {accuracy - accuracy_with_noise:<15.4f}")
print(f"{'Precision':<20} {precision_with_noise:<15.4f} {precision:<15.4f} {precision - precision_with_noise:<15.4f}")
print(f"{'Recall':<20} {recall_with_noise:<15.4f} {recall:<15.4f} {recall - recall_with_noise:<15.4f}")


Performance Comparison:
Metric               With Noise      Without Noise   Difference     
------------------------------------------------------------
Accuracy             1.0000          1.0000          0.0000         
Precision            1.0000          1.0000          0.0000         
Recall               1.0000          1.0000          0.0000         


In [40]:
# Visualize comparison
metrics = ['Accuracy', 'Precision', 'Recall']
with_noise = [accuracy_with_noise, precision_with_noise, recall_with_noise]
without_noise = [accuracy, precision, recall]

plt.figure(figsize=(12, 8))
x = np.arange(len(metrics))
width = 0.35

plt.bar(x - width/2, with_noise, width, label='With Noise')
plt.bar(x + width/2, without_noise, width, label='Without Noise')

plt.ylabel('Score')
plt.title('Model Performance Comparison')
plt.xticks(x, metrics)
plt.legend()
plt.ylim(0, 1)

for i, v in enumerate(with_noise):
    plt.text(i - width/2, v + 0.01, f'{v:.4f}', ha='center')

for i, v in enumerate(without_noise):
    plt.text(i + width/2, v + 0.01, f'{v:.4f}', ha='center')

plt.tight_layout()
plt.savefig('performance_comparison.png')
plt.close()